# Using cuOpt's Barrier Solver

The new solver may be accessed through any of cuOpt’s existing APIs or interfaces. These include:
* Python modeling API
* C API
* AMPL
* PuLP
* CVXPY
* Julia/JuMP
* Command-line interface

This notebook shows how to use the new solver from the command-line to solve linear programs defined in MPS files:

You should first enable conda inside this notebook:

In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/download/24.11.2-1_colab/Miniforge3-colab-24.11.2-1_colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:08
🔁 Restarting kernel...


You can install cuOpt from the command-line, using conda, via the command-below. **Note: this command can take a few minutes to run**. Remove the `%%capture` line to see the progress.

In [1]:
%%capture
!conda install -c rapidsai-nightly -c conda-forge -c nvidia cuopt-server=25.10.* cuopt-sh-client=25.10.*

A large linear program defined in an MPS file may be downloaded via:


In [2]:
!wget https://plato.asu.edu/ftp/lptestset/s100.mps.bz2

--2025-10-20 16:54:06--  https://plato.asu.edu/ftp/lptestset/s100.mps.bz2
Resolving plato.asu.edu (plato.asu.edu)... 129.219.51.85
Connecting to plato.asu.edu (plato.asu.edu)|129.219.51.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6741947 (6.4M) [application/x-bzip2]
Saving to: ‘s100.mps.bz2’

s100.mps.bz2        100%[===================>]   6.43M  --.-KB/s    in 0.1s    

2025-10-20 16:54:06 (46.9 MB/s) - ‘s100.mps.bz2’ saved [6741947/6741947]



You can then solve the problem with cuOpt’s command-line interface via:



In [3]:
!cuopt_cli --method=3 s100.mps.bz2

Setting parameter method to 3
Reading file s100.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.00GHz, threads (physical/logical): 1/2, RAM: 0.76 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffc1fffffffd1cffffffeb-ffffffdaf

Solving a problem with 14733 constraints, 364417 variables (0 integers), and 1777917 nonzeros
Problem scaling:
Objective coefficents range:          [3e-03, 5e-03]
Constraint matrix coefficients range: [1e+00, 3e+04]
Constraint rhs / bounds range:        [1e+00, 9e+04]
Variable bounds range:                [0e+00, 1e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Presolve attempt to remove 80 empty cols
Folding: Number of colors exceeds threshold
Folding: Coloring aborted in 0.68 seconds
Presolve eliminated 80 variables
Presolved problem: 14733 constrain

Note that inside this Google Colab Notebok, cuOpt is running on a T4 GPU. So these solve times are not representative of what you would see on a recommend GPU such as an H100.

# Deterministic Mode
By default cuOpt uses cuDSS in non-deterministic mode. This means that cuDSS may produce a (slightly) different solution, given the same inputs. cuOpt inherits the non-determinism from cuDSS. If determinism is important for your application, you can set the parameter “cudss-deterministic”  to true. This will put cuDSS in a deterministic mode.




In [4]:
!cuopt_cli --method=3 --cudss-deterministic=t s100.mps.bz2

Setting parameter cudss_deterministic to true
Setting parameter method to 3
Reading file s100.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.00GHz, threads (physical/logical): 1/2, RAM: 0.64 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffc1fffffffd1cffffffeb-ffffffdaf

Solving a problem with 14733 constraints, 364417 variables (0 integers), and 1777917 nonzeros
Problem scaling:
Objective coefficents range:          [3e-03, 5e-03]
Constraint matrix coefficients range: [1e+00, 3e+04]
Constraint rhs / bounds range:        [1e+00, 9e+04]
Variable bounds range:                [0e+00, 1e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Presolve attempt to remove 80 empty cols
Folding: Number of colors exceeds threshold
Folding: Coloring aborted in 0.67 seconds
Presolve eliminated 8

In our testing, cuDSS is about 1.5X slower in factorization and solve time when deterministic. This translates almost directly to a slowdown in cuOpt’s barrier method.

#Folding

Linear programs often include symmetries. For example, two variables may appear in identical constraints, with identical costs and bounds. We can take advantage of these symmetries to dramatically reduce the size of the linear program. This size reduction often comes with a dramatic reduction in the solve time. This process is often referred to as folding a linear program.  Folding was developed by Grohe et. al in their 2014 paper: [Dimension Reduction via Colour Refinement](https://arxiv.org/pdf/1307.5697).

Folding can result in large reductions in run-time. But to fold a linear program, a graph representation of the problem must first be colored using the [color refinement algorithm](https://www.lics.rwth-aachen.de/global/show_document.asp?id=aaaaaaaaabbtcqu). cuOpt includes a CPU implementation of the color refinement algorithm. On some LPs, coloring this graph can take longer than solving the original linear program. So cuOpt includes heuristics to decide when to try to fold an LP and when to abort coloring. These heuristics ensure good performance on average, but you may want to enable or disable folding on your own problems.

You can control folding via the folding parameter. The default value of (-1) uses these heuristics to decide automatically to fold or not, you can turn folding off by setting the parameter to 0, or force folding on by setting the parameter to (1).

We first download an example LP that solves quickly when cuOpt automatically detects the problem can be folded.

In [5]:
!wget https://plato.asu.edu/ftp/lptestset/qap15.mps.bz2

--2025-10-20 16:58:28--  https://plato.asu.edu/ftp/lptestset/qap15.mps.bz2
Resolving plato.asu.edu (plato.asu.edu)... 129.219.51.85
Connecting to plato.asu.edu (plato.asu.edu)|129.219.51.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282733 (276K) [application/x-bzip2]
Saving to: ‘qap15.mps.bz2’

qap15.mps.bz2       100%[===================>] 276.11K  --.-KB/s    in 0.06s   

2025-10-20 16:58:28 (4.79 MB/s) - ‘qap15.mps.bz2’ saved [282733/282733]



We now run cuOpt on the LP. Note that we don't need to specify folding. cuOpt automatically attempts to fold

In [6]:
!cuopt_cli --method=3 qap15.mps.bz2

Setting parameter method to 3
Reading file qap15.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.00GHz, threads (physical/logical): 1/2, RAM: 0.91 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffc1fffffffd1cffffffeb-ffffffdaf

Solving a problem with 6330 constraints, 22275 variables (0 integers), and 94950 nonzeros
Problem scaling:
Objective coefficents range:          [2e+00, 1e+02]
Constraint matrix coefficients range: [1e+00, 1e+00]
Constraint rhs / bounds range:        [1e+00, 1e+00]
Variable bounds range:                [0e+00, 0e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Folding: Colors 8093. Refinements: 8094
Folding: Coloring time 0.03 seconds
Folding: Constructing reduced problem: 2121 constraints 5970 variables and 25800 nonzeros
Folding: time 0.04 seconds
Pres

We now solve the same LP but this time we disable folding

In [7]:
!cuopt_cli --method=3 --folding=0 qap15.mps.bz2

Setting parameter folding to 0
Setting parameter method to 3
Reading file qap15.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.00GHz, threads (physical/logical): 1/2, RAM: 0.91 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffc1fffffffd1cffffffeb-ffffffdaf

Solving a problem with 6330 constraints, 22275 variables (0 integers), and 94950 nonzeros
Problem scaling:
Objective coefficents range:          [2e+00, 1e+02]
Constraint matrix coefficients range: [1e+00, 1e+00]
Constraint rhs / bounds range:        [1e+00, 1e+00]
Variable bounds range:                [0e+00, 0e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Presolved problem: 6330 constraints 22275 variables 94950 nonzeros
Scaling matrix. Maximum column norm 5.477226e+00
Barrier solver: 6330 constraints, 22275 variables,

A solve time of 0.94 seconds with folding versus 8.23 seconds without is quite an improvement!

# Dualize
This setting controls whether cuOpt decides to solve the dual problem (instead of the primal problem) in presolve. Solving the dual can be faster, especially on problems that have more constraints than variables.

The default value of -1 uses a heuristic to decide automatically whether to solve the dual or not, you can disable dualization by setting the parameter to 0, or force dualization by setting the parameter to 1.

Let us run cuOpt barrier on the supportcase10 problem. Note that we do not need to specify dualize, cuOpt's heuristics automatically determine that the problem would benefit from solving the dual.

In [2]:
!wget https://plato.asu.edu/ftp/lptestset/supportcase10.mps.bz2

--2025-10-21 22:53:24--  https://plato.asu.edu/ftp/lptestset/supportcase10.mps.bz2
Resolving plato.asu.edu (plato.asu.edu)... 129.219.51.85
Connecting to plato.asu.edu (plato.asu.edu)|129.219.51.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2038347 (1.9M) [application/x-bzip2]
Saving to: ‘supportcase10.mps.bz2’

supportcase10.mps.b 100%[===================>]   1.94M  7.13MB/s    in 0.3s    

2025-10-21 22:53:25 (7.13 MB/s) - ‘supportcase10.mps.bz2’ saved [2038347/2038347]



In [3]:
!cuopt_cli --method=3 supportcase10.mps.bz2

Setting parameter method to 3
Reading file supportcase10.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.20GHz, threads (physical/logical): 1/2, RAM: 0.76 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffa2ffffffa0ffffff94ffffffc8-2ff

Solving a problem with 165684 constraints, 14770 variables (0 integers), and 555082 nonzeros
Problem scaling:
Objective coefficents range:          [1e+00, 1e+00]
Constraint matrix coefficients range: [1e+00, 1e+00]
Constraint rhs / bounds range:        [1e+00, 1e+00]
Variable bounds range:                [0e+00, 1e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Solving the dual
Folding: Number of colors exceeds threshold
Folding: Coloring aborted in 0.12 seconds
Presolved problem: 14770 constraints 204884 variables 603942 nonzeros
Scaling matri

Note the `Solving by dual` line in the logs. This means the solver performed dualization. We now solve the same LP but this time we disable dualization. Note this can take a long time to solve, so we set a time limit of three minutes.

In [10]:
!cuopt_cli --dualize=0 --method=3 --time-limit=180 supportcase10.mps.bz2

Setting parameter dualize to 0
Setting parameter method to 3
Setting parameter time_limit to 1.800000e+02
Reading file supportcase10.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.20GHz, threads (physical/logical): 1/2, RAM: 4.37 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffa2ffffffa0ffffff94ffffffc8-2ff

Solving a problem with 165684 constraints, 14770 variables (0 integers), and 555082 nonzeros
Problem scaling:
Objective coefficents range:          [1e+00, 1e+00]
Constraint matrix coefficients range: [1e+00, 1e+00]
Constraint rhs / bounds range:        [1e+00, 1e+00]
Variable bounds range:                [0e+00, 1e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Presolve attempt to remove 140 empty cols
Folding: Number of colors exceeds threshold
Folding: Coloring aborte

### **Ordering**
This setting controls the ordering algorithm used by cuDSS for sparse factorizations. The ordering can significantly impact solver run time.

The default setting of `-1` use a heuristic to select the best ordering. The settiong of `0` uses the `cuDSS` default ordering. The setting of `1` use the Approximate Minimum Degree ordering.

Let us run cuOpt barrier with different ordering algorithms on **set-cover-model.mps** problem. First we will run the cuDSS default ordering and inspect the solve time.

In [6]:
!wget https://plato.asu.edu/ftp/lptestset/set-cover-model.mps.bz2

--2025-10-21 23:07:10--  https://plato.asu.edu/ftp/lptestset/set-cover-model.mps.bz2
Resolving plato.asu.edu (plato.asu.edu)... 129.219.51.85
Connecting to plato.asu.edu (plato.asu.edu)|129.219.51.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73534965 (70M) [application/x-bzip2]
Saving to: ‘set-cover-model.mps.bz2’

set-cover-model.mps 100%[===================>]  70.13M  54.1MB/s    in 1.3s    

2025-10-21 23:07:11 (54.1 MB/s) - ‘set-cover-model.mps.bz2’ saved [73534965/73534965]



In [7]:
!cuopt_cli --method=3 --ordering=0 set-cover-model.mps.bz2

Setting parameter method to 3
Setting parameter ordering to 0
Reading file set-cover-model.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.20GHz, threads (physical/logical): 1/2, RAM: 1.76 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffa2ffffffa0ffffff94ffffffc8-2ff

Solving a problem with 10000 constraints, 1102008 variables (0 integers), and 20442268 nonzeros
Problem scaling:
Objective coefficents range:          [3e+05, 8e+07]
Constraint matrix coefficients range: [1e+00, 1e+00]
Constraint rhs / bounds range:        [0e+00, 1e+00]
Variable bounds range:                [0e+00, 1e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Folding: Skipping
Presolved problem: 10000 constraints 1112008 variables 20452268 nonzeros
Scaling matrix. Maximum column norm 5.916080e+00
Barrier s

We now solve the same LP with the approximate minimum degree ordering.

In [8]:
!cuopt_cli --method=3 --ordering=1 set-cover-model.mps.bz2

Setting parameter method to 3
Setting parameter ordering to 1
Reading file set-cover-model.mps.bz2
cuOpt version: 25.10.0, git hash: ab1cc2b, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.20GHz, threads (physical/logical): 1/2, RAM: 3.58 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffa2ffffffa0ffffff94ffffffc8-2ff

Solving a problem with 10000 constraints, 1102008 variables (0 integers), and 20442268 nonzeros
Problem scaling:
Objective coefficents range:          [3e+05, 8e+07]
Constraint matrix coefficients range: [1e+00, 1e+00]
Constraint rhs / bounds range:        [0e+00, 1e+00]
Variable bounds range:                [0e+00, 1e+00]

Third-party presolve is disabled, skipping
Objective offset 0.000000 scaling_factor 1.000000
Folding: Skipping
Presolved problem: 10000 constraints 1112008 variables 20452268 nonzeros
Scaling matrix. Maximum column norm 5.916080e+00
Barrier s

We see an improvement from 216 seconds to 178 seconds with using the AMD ordering on this problem.


#Presolve
cuOpt can perform other operations to reduce the size of the linear program. This process is called presolve. Currently, cuOpt links against the open-source presolver Papilo. By default, Papilo is disabled for LPs. This is because in our experiments, on average, we find it faster to solve the original problem directly, rather than applying extensive presolve reductions.

However, your linear program may benefit from enabling presolve. Let's download an LP where presolve is able to reduce the problem size.




In [ ]:
!wget https://plato.asu.edu/ftp/lptestset/woodlands09.mps.bz2

--2025-10-10 19:24:23--  https://plato.asu.edu/ftp/lptestset/woodlands09.mps.bz2
Resolving plato.asu.edu (plato.asu.edu)... 129.219.51.85
Connecting to plato.asu.edu (plato.asu.edu)|129.219.51.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7064631 (6.7M) [application/x-bzip2]
Saving to: ‘woodlands09.mps.bz2’

woodlands09.mps.bz2 100%[===================>]   6.74M  4.04MB/s    in 1.7s    

2025-10-10 19:24:25 (4.04 MB/s) - ‘woodlands09.mps.bz2’ saved [7064631/7064631]



 To enable presolve for linear programs, set the presolve option to true.

In [ ]:
!cuopt_cli --method=3 --presolve=t woodlands09.mps.bz2

Setting parameter method to 3
Setting parameter presolve to true
Running file woodlands09.mps.bz2
cuOpt version: 25.10.0, git hash: e9a4f81, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.00GHz, threads (physical/logical): 1/2, RAM: 1.10 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffe4ffffff857a00-ffffff82ffffffe

Unpresolved problem: 194599 constraints, 382147 variables, 2646003 nonzeros
Calling Papilo presolver
Disabling the presolver methods that do not support dual postsolve
Presolve status: reduced the problem
Presolve removed: 23911 constraints, 22008 variables, 586657 nonzeros
Presolved problem: 170688 constraints, 360139 variables, 2059346 nonzeros
Papilo presolve time: 1.582778
Solving a problem with 170688 constraints 360139 variables (0 integers) and 2059346 nonzeros
Objective offset 0.000000 scaling_factor 1.000000
Folding: Number of colors exceeds threshold
Fo

Note that Papilo includes presolve reductions that do not support dual post-solve. Meaning that Papilo cannot extract the optimal dual variables for the original problem from the solution of the presolved problem. If you need the dual solution to the linear program, you can set the parameter “dual-postsolve” to true. This will disable all reductions that do not support dual post-solve, and allow you to recover the dual solution. However, the presolved linear program may be larger in this case.

In [ ]:
!cuopt_cli --method=3 --presolve=t --dual-postsolve=t woodlands09.mps.bz2

Setting parameter dual_postsolve to true
Setting parameter method to 3
Setting parameter presolve to true
Running file woodlands09.mps.bz2
cuOpt version: 25.10.0, git hash: e9a4f81, host arch: x86_64, device archs: 70-real,75-real,80-real,86-real,90a-real,100f-real,120a-real,120
CPU: Intel(R) Xeon(R) CPU @ 2.00GHz, threads (physical/logical): 1/2, RAM: 1.11 GiB
CUDA 12.9, device: Tesla T4 (ID 0), VRAM: 14.74 GiB
CUDA device UUID: ffffffe4ffffff857a00-ffffff82ffffffe

Unpresolved problem: 194599 constraints, 382147 variables, 2646003 nonzeros
Calling Papilo presolver
Disabling the presolver methods that do not support dual postsolve
Presolve status: reduced the problem
Presolve removed: 23911 constraints, 22008 variables, 586657 nonzeros
Presolved problem: 170688 constraints, 360139 variables, 2059346 nonzeros
Papilo presolve time: 1.608108
Solving a problem with 170688 constraints 360139 variables (0 integers) and 2059346 nonzeros
Objective offset 0.000000 scaling_factor 1.000000
Foldi